In [1]:
import geopandas as gpd

stops_gdf = gpd.read_file("stopsLO.geojson")
stops_gdf.rename(columns={"Автоб": "route"}, inplace=True)
stops_gdf.dropna(subset="route", axis=0, inplace=True, how="any")
stops_gdf


,name,route,geometry
0,Дусьево,"593,593/596,847,856,860,860Л,864,865,865Д,867,...",POINT (31.72832 59.93558)
1,None,247,POINT (32.44568 59.28601)
2,Горная Шальдиха,"588,589,590,593,593/594",POINT (31.46034 59.87465)
3,Мучихино,590,POINT (31.52410 59.86666)
4,Васильково,590,POINT (31.58237 59.87627)
...,...,...,...
8386,\nЗаостровье,94,POINT (33.27731 60.62779)
8387,\nЗаостровье,94,POINT (33.27741 60.62765)
8388,Горка,94,POINT (33.22445 60.63948)
8389,Новая Слобода,"89А,91",POINT (33.33512 60.78394)


In [2]:
import pandas as pd
from tqdm.auto import tqdm

tqdm.pandas()
unique_routes = set()
for index, row in (stops_gdf.iterrows()):
    for i in str(row["route"]).replace('.', ',').rstrip().lstrip().split(","):
        if i != "":
            unique_routes.add(i.lstrip().rstrip())
unique_routes = pd.DataFrame(index=list(unique_routes))
unique_routes["geometry"] = [[]] * len(unique_routes)
for index, row in (stops_gdf.iterrows()):
    for i in str(row["route"]).replace('.', ',').rstrip().lstrip().split(","):
        if i != "":
            unique_routes.loc[i.lstrip().rstrip(), "geometry"] = unique_routes.loc[i.lstrip().rstrip()]["geometry"] + [
                row["geometry"]]

unique_routes

,geometry
2342,"[POINT (29.9698833 60.7031999), POINT (30.1937..."
475,"[POINT (30.9893514 59.887793), POINT (30.99584..."
860Д,"[POINT (32.1405498 60.0080918), POINT (32.7433..."
87,"[POINT (33.5826273 60.7344281), POINT (33.5820..."
4а,"[POINT (28.1500398 59.0951286), POINT (28.1686..."
...,...
107с,"[POINT (28.098742562071617 59.12618473418365),..."
53/54,"[POINT (28.5917039 59.4084047), POINT (28.6231..."
24,"[POINT (32.319827158230765 59.92887468138372),..."
3,"[POINT (33.5536502 59.6433943), POINT (32.3536..."


In [3]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
from shapely import Point


def clusterize(loc) -> pd.DataFrame:
    data = np.array([[p.x, p.y] for p in loc["geometry"]])
    noise = np.array([31.212390, 58.611219])
    data = np.append(data, [noise], axis=0)

    best_silhouette = -1
    best_labels = None
    k_values = range(2, 10 if len(data) > 10 else 2)

    for k in k_values:
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(data)
        labels = kmeans.labels_
        silhouette = silhouette_score(data, labels)
        if silhouette > best_silhouette:
            best_silhouette = silhouette
            best_labels = labels

    data = data[:-1]
    if best_labels is not None:
        best_labels = best_labels[:-1]

    if best_silhouette < 0.93:
        best_labels = [0 for _ in range(0, len(data))]
    data = pd.DataFrame({
        'label': best_labels,
        'geometry': [Point(p[0], p[1]) for p in data]
    })
    grouped_data = data.groupby('label')['geometry'].apply(list)
    grouped_data = pd.DataFrame(grouped_data)
    grouped_data.index = grouped_data.index + 1
    return grouped_data



In [5]:
devided_routes = pd.DataFrame()
basic_routes = pd.DataFrame()
for index, row in tqdm(unique_routes.iterrows(), total=unique_routes.shape[0]):
    clusterized = clusterize(row)
    if clusterized.shape[0] > 1:
        clusterized.index = index + "__" + clusterized.index.astype(str)
        devided_routes = pd.concat([devided_routes, clusterized])
    else:
        clusterized.index = [index]
        basic_routes = pd.concat([basic_routes, clusterized])
devided_routes

  0%|          | 0/593 [00:00<?, ?it/s]

,geometry
label,
23__1,"[POINT (29.0938556 59.9105085), POINT (29.0855..."
23__2,"[POINT (32.3004009 59.9837254), POINT (32.2995..."
23__3,"[POINT (29.5985086 60.2578632), POINT (29.5748..."
4__1,"[POINT (30.647734081369755 59.99128877944062),..."
4__2,"[POINT (34.1730592 59.5150043), POINT (34.1663..."
...,...
3__2,"[POINT (29.0855075 59.896653), POINT (29.08416..."
3__3,"[POINT (32.3536583 59.8943022), POINT (32.3623..."
3__4,"[POINT (29.847835 58.7284453), POINT (29.84985..."


In [132]:
row = devided_routes.loc["8__1"]
city_crs = 32636
geometry_list = row["geometry"]
test_gdf = gpd.GeoDataFrame(data={"label": ["8__1" for _ in geometry_list], "geometry": geometry_list})
test_gdf = test_gdf.set_crs(4326)
test_gdf = test_gdf.to_crs(city_crs)
test_gdf = test_gdf.reset_index()



project stops and create new graph

In [145]:
import networkx as nx
import osmnx as ox
from shapely import from_wkt, LineString, Point
from IPython.display import display

nx_graph = nx.read_graphml("graph_for_test.graphml")
for i in nx_graph.edges(data=True):
    i[2]['geometry'] = from_wkt(str(i[2]['geometry']))
gdf_nodes,gdf_edges = ox.graph_to_gdfs(nx_graph)
gdf_edges: gpd.GeoDataFrame = gdf_edges.reset_index()
gdf_buffer = gdf_edges.copy()
gdf_buffer["geometry"] = gdf_buffer["geometry"].apply(lambda x:LineString(x).buffer(-3,single_sided=True))
join = gpd.sjoin_nearest(test_gdf, gdf_buffer, distance_col="dist", how="inner")
join

,index,label,geometry,index_right,u,v,key,length_meter,type,time_min,dist
0,0,8__1,POINT (316646.610 6514483.163),370,63,42,0,200.473,car,0.71,3.806132
1,1,8__1,POINT (317519.412 6514106.930),32,2,150,0,236.390,car,0.83,4.829858
2,2,8__1,POINT (317663.599 6514682.084),202,19,4,0,184.469,car,0.65,3.319083
3,3,8__1,POINT (317656.452 6515037.918),234,314,8,0,112.972,car,0.40,14.377155
4,4,8__1,POINT (317817.879 6515489.241),87,18,22,0,372.057,car,1.31,1.010399
5,5,8__1,POINT (317431.130 6514889.561),64,5,219,0,131.615,car,0.46,5.770847
6,6,8__1,POINT (317328.323 6514502.463),70,219,149,0,614.199,car,2.17,8.316723
7,7,8__1,POINT (317246.746 6514211.284),926,149,1,0,232.238,car,0.82,8.156357
8,8,8__1,POINT (317505.902 6515612.571),219,21,274,0,177.763,car,0.63,25.440546
9,9,8__1,POINT (316028.091 6515690.731),256,26,360,0,362.026,car,1.28,4.566709


In [146]:

points = pd.DataFrame(data=None)
for index, row in join.iterrows():
    point: Point = row["geometry"]
    edges_to_delete,start,end = row[["index_right","u","v"]]
    line: LineString = gdf_edges.loc[edges_to_delete, "geometry"]
    nearest_point = line.interpolate(line.project(point))
    points = pd.concat(
        [points, pd.DataFrame({"index": [index], "edge_to_del": [edges_to_delete],"start": [start],"end": [end], "geometry": [nearest_point]})])
points = points.set_index("index")
points


,edge_to_del,start,end,geometry
index,,,,
0,370,63,42,POINT (316651.07651155966 6514478.026966341)
1,32,2,150,POINT (317511.8248325282 6514108.866492845)
2,202,19,4,POINT (317657.4653274753 6514683.602697591)
3,234,314,8,POINT (317640.1687647791 6515043.985640216)
4,87,18,22,POINT (317814.1559538966 6515490.733136549)
5,64,5,219,POINT (317439.6073455325 6514887.31000488)
6,70,219,149,POINT (317339.3060203759 6514499.734664751)
7,926,149,1,POINT (317257.50780811923 6514208.341981867)
8,219,21,274,POINT (317533.25234776846 6515604.773159838)


In [152]:
from networkx import NetworkXError

new_graph: nx.MultiDiGraph = nx_graph.copy()
for index,row in points.iterrows():
    try:
        u,v = row[["start","end"]]
        new_graph.remove_edge(u,v)
    except NetworkXError:
        pass
    
new_graph.add_node('0')
new_graph.add_node('0')
new_graph.add_node('0')
new_graph.add_node('0')
new_graph.nodes

NodeView(('0', '74', '264', '41', '137', '1', '291', '262', '151', '2', '177', '150', '316', '3', '246', '155', '4', '169', '168', '9', '5', '24', '219', '6', '121', '8', '232', '7', '274', '18', '519', '512', '191', '10', '187', '110', '413', '182', '11', '374', '295', '12', '104', '14', '76', '13', '93', '276', '188', '351', '15', '165', '501', '218', '47', '16', '240', '166', '17', '158', '458', '363', '46', '22', '410', '19', '37', '20', '440', '324', '443', '21', '225', '315', '88', '220', '23', '314', '255', '25', '96', '184', '470', '317', '26', '357', '360', '30', '27', '250', '29', '28', '251', '31', '32', '209', '33', '208', '34', '164', '163', '161', '207', '35', '156', '36', '38', '72', '533', '42', '132', '511', '39', '40', '139', '73', '531', '66', '138', '63', '43', '62', '44', '58', '57', '45', '52', '51', '142', '154', '206', '153', '308', '193', '48', '55', '466', '49', '339', '50', '508', '56', '141', '53', '143', '54', '59', '265', '94', '234', '61', '64', '60', '65

In [141]:
for index,row in points.iterrows():
    
    

SyntaxError: incomplete input (3417650051.py, line 3)

In [13]:
import networkit as nk
from src.dongraphio.utils import matrix_utils

mobility_sub_graph = matrix_utils.get_subgraph(
    nx_graph.copy(),
    "type",
    ["car"],
)

nk_graph = matrix_utils.convert_nx2nk(
    mobility_sub_graph, idmap=matrix_utils.get_nx2nk_idmap(mobility_sub_graph), weight="length_meter"
)

graph_with_geom = matrix_utils.load_graph_geometry(mobility_sub_graph)
df = pd.DataFrame.from_dict(dict(graph_with_geom.nodes(data=True)), orient="index")
graph_gdf = gpd.GeoDataFrame(df, geometry=df["geometry"], crs=city_crs)
nearest_nodes = graph_gdf["geometry"].sindex.nearest(
    test_gdf["geometry"], return_distance=True, return_all=False
)

distance_matrix = pd.DataFrame(0, index=nearest_nodes[0][1], columns=nearest_nodes[0][1])
for source in tqdm(nearest_nodes[0][1], total=len(nearest_nodes[0][1])):
    for dest in nearest_nodes[0][1]:
        heu = None
        r = nk.distance.AStar(G=nk_graph, heu=heu, source=source, target=dest).run()


KeyboardInterrupt: 

In [ ]:
from shapely.geometry import MultiPoint

devided_routes["geometry"] = devided_routes["geometry"].apply(lambda x: MultiPoint(x))
devided_routes = gpd.GeoDataFrame(data=devided_routes, geometry="geometry")
devided_routes.to_file("new_routes.geojson")


In [6]:
from shapely import from_wkt
from src.dongraphio.utils import get_osmnx_graph
import osmnx as ox
import networkx as nx

G_drive: nx.MultiDiGraph = get_osmnx_graph(
    2705630, 32636, "drive", truncate_by_edge=False
)
nx.write_graphml(G_drive, "graph_for_test.graphml")
for i in G_drive.edges(data=True):
    i[2]['geometry'] = from_wkt(str(i[2]['geometry']))
gdf = ox.graph_to_gdfs(G_drive, nodes=False)
gdf.to_file("graph_for_test.geojson")

2024-03-18 15:58:27.231 | DEBUG    | src.dongraphio.utils.graphs:get_osmnx_graph:58 - Extracting and preparing drive graph from OSM ...
